In [128]:
from IPython.core.display import HTML
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
from pandas import DataFrame
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [129]:
holidays2020 = ['2020-01-01', '2020-01-20', '2020-02-17', '2020-04-10', '2020-05-25', '2020-07-03', '2020-09-07']
holidays2021 = ['2021-01-01', '2021-01-18', '2021-02-15', '2021-04-02', '2021-05-31', '2021-07-05', '2021-09-06', '2021-11-25', '2021-12-24']
holidays2022 = ['2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30', '2022-06-20', '2022-07-04', '2022-09-05', '2022-12-26']

weekdays_year = pd.date_range(start='2022/1/1', end='2022/12/31', freq='B')

from datetime import date





def closestTradingDate():
    # closest trading day
    from pandas.tseries.offsets import DateOffset

    today_date = pd.Timestamp(date.today())
    closest_date = today_date

    # Checks if today's date is SATURDAY
    if (today_date.isoweekday() == 6):
        closest_date = today_date - DateOffset(days = 1)
    # Checks if today_date is SUNDAY

    elif (today_date.isoweekday() == 7):
        closest_date = today_date - DateOffset(days = 2)

    # Check if the Friday is a holiday and subtract another day
    while(str(closest_date)[:10] in holidays2022):
        closest_date = closest_date - DateOffset(days = 1)
    
    # returns only date ('YYYY-MM-DD') without time
    closest_date = str(closest_date)[:10]
    
    return closest_date


def listOfDatesToCurrent():
# returns a list of active trading dates from the year 2022 (can change to be any year)
    dates = []

    for num in range(len(weekdays_year)):

        # Changing it from a 'pandas timestamp' to a string with only year-month-day
        dates.append(weekdays_year.date[num].strftime('%Y-%m-%d'))

    # Disclude holidays from active trading days
    dates = [date for date in dates if date not in holidays2022]
    try:
        # this slice will stop at today's date from first day
        # finds the index of today's date (date.today()) and slices list to stop at today
        dates_to_present = dates[:dates.index(closestTradingDate())+1]
    except:
        print("ERROR?! UNCOMMENT BELOW LINE TO GIVE STATIC AMOUNT OF DATES")
    #     dates_to_present = dates[:134]

    return dates_to_present
# closestTradingDate()
# listOfDatesToCurrent()

In [156]:
class TwoDateTicker():
    def __init__(self, ticker, date1, date2):
        
        # make 'date1' always the closer date
        datelist = list(reversed(listOfDatesToCurrent()))
        if datelist.index(date1) > datelist.index(date2):
            temp = date1
            date1 = date2
            date2 = temp
            
        self.ticker = ticker
        self.date1 = date1
        self.date2 = date2
        self.path = r"C:\Users\lorig\Desktop\PythonCode\Projects, Exercises, Programs\Trade Alert Journey\Data\{}\{}_oi_strike_series".format(self.ticker, self.ticker)
        self.filenameList = os.listdir(self.path)
        
        # Might not use these?
        self.df1, self.df2 = self.dates()
        
        
        
    def dates(self):
        date_1 = False
        date_2 = False
        df1 = 0
        df2 = 0
#         print("Enter two dates - I will see whether they are in our system.")
        for file in self.filenameList:
            x = file.split("_")[4][:-4]
            if self.date1 == x:
                date_1 = True
                df1 = pd.read_csv("{}/{}".format(self.path, file))
            if self.date2 == x:
                date_2 = True
                df2 = pd.read_csv("{}/{}".format(self.path, file))

        if date_1 and date_2 is True:
            print('Data is available for these dates.\n')

        else:
            printmd("***Please enter new dates!***")
        self.df1 = df1
        return df1, df2
    
    def print(self):
        display(self.df1.head(3))
        
    
    def difference(self):
        df1, df2 = self.dates()
        
        

In [157]:
x = TwoDateTicker('SPY', '2022-08-19', '2022-08-25')


x.print()


Data is available for these dates.



,Strikes,2022-08-26,2022-08-29,2022-08-31,2022-09-02,2022-09-06,2022-09-07,2022-09-09,2022-09-12,2022-09-14,...,2023-03-17,2023-03-31,2023-06-16,2023-06-30,2023-09-15,2023-12-15,2024-01-19,2024-06-21,2024-12-20,Total OI per Strike
0,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27109.0
1,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8141.0
2,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5012.0


In [120]:
class Folder():
    def __init__(self, foldername):
        self.foldername = foldername
        self.filenameList = os.listdir(self.foldername)
        

    def ordered_dates(self, date1, date2):
        datelist = list(reversed(listOfDatesToCurrent()))
        if datelist.index(date1) > datelist.index(date2):
            temp = date1
            date1 = date2
            date2 = temp
        return date1, date2
            
    def dates(self, date1, date2):
        date_1 = False
        date_2 = False
        df1 = 0
        df2 = 0
        date1, date2 = self.ordered_dates(date1, date2)
        
#         print("Enter two dates - I will see whether they are in our system.")
        for file in self.filenameList:
            x = file.split("_")[4][:-4]
            if date1 == x:
                date_1 = True
                df1 = pd.read_csv("{}/{}".format(self.foldername, file))
            if date2 == x:
                date_2 = True
                df2 = pd.read_csv("{}/{}".format(self.foldername, file))
                
        if date_1 and date_2 is True:
            print('Data is available for these dates.\n')
            
        else:
            printmd("***Please enter new dates!***")
            
        return df1, df2
    
        # if i have the data, df1 and df2 should now have a pandasDataFrame obj
        
    def difference(self):
        df1, df2 = self.dates()
        
        
        
    def expiries(self):
        expiries_list = []
#         expiries_df1 = ?
#         expiries_df2 = ?
        df1, df2 = self.dates()
        
        for i in range(len(df1.count())):
            x = df.iloc[0][i]
            y = expiryRegex.findall(x)
            if bool(y) == True:
                for item in y:
                    expiries_list.append(item)
        
        

In [99]:
# first I need to write the regexes to turn the HTML into excel files.
# i will start writing code that will process some data in the files.

In [100]:
# I should take two dates and ticker as input like "2022-06-03" & "2022-07-04"
# then my program should go to the folder and select the two xlsx files 
# to find the total OI change between the two

# use regexes or .split() to find the right files from their numerical dates

In [121]:
import os
import pandas as pd

main_data_folder = r"C:\Users\lorig\Desktop\PythonCode\Projects, Exercises, Programs\Trade Alert Journey\Data"
ticker = "AAPL\AAPL_oi_strike_series"

u = Folder(r"{}\{}".format(main_data_folder, ticker))
df1, df2 = u.dates('2022-07-25', '2022-08-15')

Data is available for these dates.



In [119]:
x, y = u.dates("2022-08-04", "2022-07-14")
x =x.set_index('Strikes')


both dates exist in system


In [117]:
x['2022-08-05'][35]

0.0

TypeError: dates() missing 2 required positional arguments: 'date1' and 'date2'

In [90]:
df = pd.DataFrame(0)

ValueError: DataFrame constructor not properly called!